In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
#warning.filterwarning("ignore")

import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('../input/train.csv')

In [ ]:
test = pd.read_csv('../input/test.csv')

In [ ]:
train.info()
test.info()

In [ ]:
#converting floats to int as logres can't take in continuous variables 
from sklearn import utils 
from sklearn import preprocessing

lab_train = preprocessing.LabelEncoder()
train[['walkDistance']] = lab_train.fit_transform(train[['walkDistance']])
train[['damageDealt']] = lab_train.fit_transform(train[['damageDealt']])
train[['longestKill']] = lab_train.fit_transform(train[['longestKill']])
train[['rideDistance']] = lab_train.fit_transform(train[['rideDistance']])
train[['swimDistance']] = lab_train.fit_transform(train[['swimDistance']])
train[['winPlacePerc']] = lab_train.fit_transform(train[['winPlacePerc']])

test[['walkDistance']] = lab_train.fit_transform(test[['walkDistance']])
test[['damageDealt']] = lab_train.fit_transform(test[['damageDealt']])
test[['longestKill']] = lab_train.fit_transform(test[['longestKill']])
test[['rideDistance']] = lab_train.fit_transform(test[['rideDistance']])
test[['swimDistance']] = lab_train.fit_transform(test[['swimDistance']])

train.info()

In [ ]:
print('Training data shape: ', train.shape)
train.head()


train['winPlacePerc'].value_counts()

In [ ]:
train['winPlacePerc'].plot.hist()

In [ ]:
train['walkDistance'].describe()

In [ ]:
train_labels = train['winPlacePerc']

# Align the training and testing data, keep only columns present in both dataframes
train, test = train.align(test, join = 'inner', axis = 1)

# Add the target back in
train['winPlacePerc'] = train_labels

print('Training Features shape: ', train.shape)
print('Testing Features shape: ', test.shape)

In [ ]:
correlations = train.corr()['winPlacePerc'].sort_values()

print(correlations)

In [ ]:
plt.style.use('fivethirtyeight')

plt.hist(train['winPlacePerc'], edgecolor = 'k', bins =25)
plt.title('Walk Distance'); plt.xlabel('Meters'); plt.ylabel('Count');

In [ ]:
cor_data = train[['winPlacePerc', 'walkDistance', 'killPlace','boosts','weaponsAcquired']]
cor_data_cors = cor_data.corr()
cor_data_cors

In [ ]:
plt.figure(figsize = (8,6))

sns.heatmap(cor_data_cors, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = True, vmax = 0.6)


In [ ]:
#polynomial features 

poly_features = train[['walkDistance', 'killPlace','boosts','weaponsAcquired','winPlacePerc']]
poly_features_test = test[['walkDistance', 'killPlace','boosts','weaponsAcquired']]


In [ ]:
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')
poly_target = poly_features['winPlacePerc']

poly_features = poly_features.drop(columns = ['winPlacePerc'])

# Need to impute missing values
poly_features = imputer.fit_transform(poly_features)
poly_features_test = imputer.transform(poly_features_test)
from sklearn.preprocessing import PolynomialFeatures
poly_transformer = PolynomialFeatures(degree = 3)

In [ ]:
poly_transformer.fit(poly_features)

poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

In [ ]:
poly_transformer.get_feature_names(input_features = ['walkDistance', 'killPlace','boosts','weaponsAcquired'])[:15]

In [ ]:
poly_features = pd.DataFrame(poly_features, 
                             columns = poly_transformer.get_feature_names(['walkDistance', 'killPlace','boosts','weaponsAcquired']))

# Add in the target
poly_features['winPlacePerc'] = poly_target

# Find the correlations with the target
poly_corrs = poly_features.corr()['winPlacePerc'].sort_values()

# Display most negative and most positive
print(poly_corrs.head(10))
print(poly_corrs.tail(5))

In [ ]:
poly_features_test = pd.DataFrame(poly_features_test, 
                             columns = poly_transformer.get_feature_names
                             (['walkDistance', 'killPlace','boosts','weaponsAcquired']))

poly_features['Id'] = train['Id']
train_poly = train.merge(poly_features, on = 'Id', how = 'left')

# Merge polnomial features into testing dataframe
poly_features_test['Id'] = test['Id']
test_poly = test.merge(poly_features_test, on = 'Id', how = 'left')

# Align the dataframes
train_poly, test_poly = train_poly.align(test_poly, join = 'inner', axis = 1)

# Print out the new shapes
print('Training data with polynomial features shape: ', train_poly.shape)
print('Testing data with polynomial features shape:  ', test_poly.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, Imputer

# Drop the target from the training data
if 'winPlacePerc' in train:
    test_train = train.drop(columns = ['winPlacePerc'])
else:
    test_train = train.copy()
    
# Feature names
features = list(test_train.columns)

# Copy of the testing data
test_test = test.copy()

# Median imputation of missing values
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(test_train)

# Transform both training and testing data
test_train = imputer.transform(test_train)
test_test = imputer.transform(test)

# Repeat with the scaler
scaler.fit(test_train)
test_train = scaler.transform(test_train)
test_test = scaler.transform(test_test)

print('Training data shape: ', test_train.shape)
print('Testing data shape: ', test_test.shape)

In [ ]:
test.info()

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C=0.0001)

log_reg.fit(test_train, train_labels)